# Customer Purchase Prediction

This notebook demonstrates a machine learning project to predict whether a customer will purchase a product based on features like time on site, pages viewed, device, and referral source.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

## Data Generation

We will generate synthetic data for demonstration purposes. The features are:
- time_on_site: continuous, minutes spent on site
- pages_viewed: integer, number of pages viewed
- device: categorical, 'mobile', 'desktop', 'tablet'
- referral_source: categorical, 'google', 'facebook', 'direct', 'email'

Target: purchase (0 or 1)

In [4]:
# Generate synthetic data
n_samples = 1000

# Features
time_on_site = np.random.exponential(5, n_samples)  # exponential distribution
pages_viewed = np.random.poisson(3, n_samples) + 1  # poisson +1 to avoid 0
device = np.random.choice(['mobile', 'desktop', 'tablet'], n_samples)
referral_source = np.random.choice(['google', 'facebook', 'direct', 'email'], n_samples)

# Target: purchase probability depends on features
purchase_prob = 0.1 + 0.01 * time_on_site + 0.05 * pages_viewed
purchase_prob += np.where(device == 'desktop', 0.1, 0)
purchase_prob += np.where(referral_source == 'google', 0.05, 0)
purchase_prob = np.clip(purchase_prob, 0, 1)
purchase = np.random.binomial(1, purchase_prob)

# Create DataFrame
data = pd.DataFrame({
    'time_on_site': time_on_site,
    'pages_viewed': pages_viewed,
    'device': device,
    'referral_source': referral_source,
    'purchase': purchase
})

print(data.head())
print(data.describe())

   time_on_site  pages_viewed   device referral_source  purchase
0      4.238813             1  desktop        facebook         0
1      0.237570             3  desktop          direct         1
2      4.890292             4  desktop          direct         0
3      4.918809             4   tablet          google         0
4     11.091256             4  desktop           email         1
       time_on_site  pages_viewed     purchase
count   1000.000000   1000.000000  1000.000000
mean       5.177034      3.975000     0.414000
std        4.963842      1.741093     0.492795
min        0.027066      1.000000     0.000000
25%        1.586782      3.000000     0.000000
50%        3.721620      4.000000     0.000000
75%        7.368239      5.000000     1.000000
max       41.298671     12.000000     1.000000


## Data Preprocessing

Handle categorical variables using label encoding and scale numerical features.

In [5]:
# Preprocessing
# Encode categorical variables
le_device = LabelEncoder()
le_referral = LabelEncoder()

data['device_encoded'] = le_device.fit_transform(data['device'])
data['referral_encoded'] = le_referral.fit_transform(data['referral_source'])

# Features and target
X = data[['time_on_site', 'pages_viewed', 'device_encoded', 'referral_encoded']]
y = data['purchase']

# Scale numerical features
scaler = StandardScaler()
X[['time_on_site', 'pages_viewed']] = scaler.fit_transform(X[['time_on_site', 'pages_viewed']])

print(X.head())

   time_on_site  pages_viewed  device_encoded  referral_encoded
0     -0.189106     -1.709551               0                 2
1     -0.995587     -0.560273               0                 0
2     -0.057795      0.014366               0                 0
3     -0.052047      0.014366               2                 3
4      1.192057      0.014366               0                 1


In [6]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (800, 4), Test shape: (200, 4)


## Model Training

Train Logistic Regression and Random Forest models.

In [7]:
# Logistic Regression
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

print("Models trained.")

Models trained.


## Model Evaluation

Evaluate models on test data using accuracy and precision.

In [8]:
# Predictions
y_pred_log = log_reg.predict(X_test)
y_pred_rf = rf.predict(X_test)

# Metrics
acc_log = accuracy_score(y_test, y_pred_log)
prec_log = precision_score(y_test, y_pred_log)

acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf)

print(f"Logistic Regression - Accuracy: {acc_log:.2f}, Precision: {prec_log:.2f}")
print(f"Random Forest - Accuracy: {acc_rf:.2f}, Precision: {prec_rf:.2f}")

Logistic Regression - Accuracy: 0.64, Precision: 0.60
Random Forest - Accuracy: 0.58, Precision: 0.48


## Conclusion

This notebook demonstrated the basics of a customer purchase prediction project, including data generation, preprocessing, model training with Logistic Regression and Random Forest, and evaluation with accuracy and precision metrics. In a real project, you would use actual data and perform hyperparameter tuning, cross-validation, and more advanced evaluation.